In [ ]:
from scipy.sparse import csc_matrix
import trimesh
import skeletor as sk
from util.mesh import MeshProcessor
#mesh = trimesh.load("/Users/ackermand/Documents/Downloads/410_roi1.obj")
mp = MeshProcessor(path = "https://janelia-cosem-datasets.s3.amazonaws.com/jrc_hela-2/neuroglancer/mesh/mito_seg",lod = 2, min_branch_length = 20)
ms = mp.get_mesh(id=45)
trimesh_mesh = trimesh.Trimesh(ms.current_mesh().vertex_matrix(), ms.current_mesh().face_matrix())

In [ ]:
trimesh_mesh.show()

In [ ]:
#mesh = sk.pre.fix_mesh(mesh, remove_disconnected=5, inplace=False)
mesh = sk.pre.contract(trimesh_mesh)

In [ ]:
skel = sk.skeletonize.by_tangent_ball(trimesh_mesh)# 
# skel = sk.skeletonize.by_wavefront(trimesh_mesh, waves=1, step_size=2)
#skel = sk.skeletonize.by_teasar(trimesh_mesh, inv_dist = 100)
skel = sk.post.clean_up(skel)


In [ ]:
import navis
import numpy as np
from navis import graph
n = navis.TreeNeuron(skel, soma=None)
pruned = navis.prune_twigs(n,100)
seg_lengths_before = np.array([graph.segment_length(n, s) for s in n.segments])
seg_lengths_after = np.array([graph.segment_length(pruned, s) for s in pruned.segments])

print(seg_lengths_before,seg_lengths_after)
skel2 = sk.Skeleton(pruned.nodes[['node_id', 'parent_id', 'x', 'y', 'z', 'radius']],mesh=trimesh_mesh)
print(n.n_leafs,pruned.n_leafs)
skel2 = sk.post.clean_up(skel2)
n2 = navis.TreeNeuron(skel2, soma=None)
longest_path = navis.longest_neurite(n2, from_root=False)
navis.plot3d([n2,longest_path,trimesh_mesh])

In [ ]:
skel.show(mesh=True)


In [ ]:
import numpy as np
import time
metric = np.random.random(len(mesh.vertices))
t = time.time()
face_metrics = np.mean(metric[mesh.faces.flatten()].reshape((-1,3)),axis=1)


In [ ]:

import numpy as np
import time
e = np.array(mesh.edges)
t = time.time()
temp = csc_matrix((np.ones(len(e)),(e[:,0],e[:,1])), shape=(len(mesh.vertices),len(mesh.vertices)))
print(time.time()-t)
t = time.time()
for i in range(10):
    temp = temp.dot(temp)
print(type(temp))
print(time.time()-t)
rows,cols = temp.nonzero()
cols[rows==0]

In [ ]:
g = nx.from_edgelist(mesh.edges_unique)

In [ ]:
a = dict(nx.all_pairs_shortest_path_length(g,cutoff=5))

In [ ]:
a[0][0]

In [ ]:
import pyglet
window = pyglet.window.Window()
label = pyglet.text.Label('Hello, world',
                          font_name='Times New Roman',
                          font_size=36,
                          x=window.width//2, y=window.height//2,
                          anchor_x='center', anchor_y='center')

@window.event
def on_draw():
    window.clear()
    label.draw()

pyglet.app.run()

In [ ]:
import numpy as np
a = np.arange(10)
a.clip(4,7)
print(a)

In [ ]:
import pymeshlab
import numpy as np
ms = pymeshlab.MeshSet()
ms.load_new_mesh("/Users/ackermand/Documents/Downloads/410_roi1.obj")
# ms.load_new_mesh("./content/MPI-FAUST/meshes/tr_reg_000.ply")
mesh = ms.current_mesh()
t = time.time()
ms.meshing_repair_non_manifold_edges()
t=time.time()
ms.compute_scalar_by_geodesic_distance_from_given_point_per_vertex(np.array[mesh.vertex_matrix()[0,:]])
print(time.time()-t)

In [ ]:
print(mesh.vertex_matrix()[0,:])
t=time.time()
ms.compute_scalar_by_geodesic_distance_from_given_point_per_vertex(startpoint = mesh.vertex_matrix()[0,:],maxdistance=pymeshlab.Percentage(1))
print(time.time()-t)

In [ ]:
from util.mesh import MeshProcessor
from tqdm import tqdm
import numpy as np

mp = MeshProcessor(path = "https://janelia-cosem-datasets.s3.amazonaws.com/jrc_hela-2/neuroglancer/mesh/mito_seg",lod = 2, min_branch_length = 20, )
#mp.process_mesh(id=53994)
#mp.process_mesh(id=84602)
#mp.process_mesh(id=322)
#mp.process_mesh(id=116)
momenta = np.zeros((421,3),dtype=np.float32)
for i in tqdm(range(1,422)):
    momenta[i-1,:] = mp.process_mesh(id=i)




In [3]:
from util.mesh import MeshProcessor
from tqdm import tqdm
import numpy as np
import dask
from dask.distributed import Client, progress
import pandas as pd

client = Client(threads_per_worker=2, n_workers=1)
client.cluster.scale(8)
print(client.dashboard_link)
mp = MeshProcessor(path = "https://janelia-cosem-datasets.s3.amazonaws.com/jrc_mus-liver/neuroglancer/mesh/mito_seg",lod = 0, min_branch_length = 100, use_skeletons=True)
#momenta = np.zeros((421,3),dtype=np.float32)
lazy_results = []
for i in range(1,87077,100):
    lazy_results.append(mp.process_mesh(id=i))
results = dask.compute(*lazy_results)
df = pd.DataFrame.from_records(results)
display(df)

/Users/ackermand/miniconda3/envs/mesh_playground/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 61611 instead
  f"Port {expected} is already in use.\n"


http://127.0.0.1:61611/status


/Users/ackermand/miniconda3/envs/mesh_playground/lib/python3.7/site-packages/trimesh/util.py:128: RuntimeWarning: invalid value encountered in reciprocal
  norm[valid] **= -1
/Users/ackermand/miniconda3/envs/mesh_playground/lib/python3.7/site-packages/trimesh/util.py:128: RuntimeWarning: invalid value encountered in reciprocal
  norm[valid] **= -1
/Users/ackermand/miniconda3/envs/mesh_playground/lib/python3.7/site-packages/trimesh/util.py:128: RuntimeWarning: invalid value encountered in reciprocal
  norm[valid] **= -1
/Users/ackermand/miniconda3/envs/mesh_playground/lib/python3.7/site-packages/trimesh/util.py:128: RuntimeWarning: invalid value encountered in reciprocal
  norm[valid] **= -1
/Users/ackermand/miniconda3/envs/mesh_playground/lib/python3.7/site-packages/trimesh/util.py:128: RuntimeWarning: invalid value encountered in reciprocal
  norm[valid] **= -1


,id,volume,surface_area,principal_inertia_component_0,principal_inertia_component_normalized_0,principal_inertia_component_1,principal_inertia_component_normalized_1,principal_inertia_component_2,principal_inertia_component_normalized_2,num_fragments,num_branches,longest_path
0,1,4.273944e+08,2.858187e+06,3.185847e+13,0.273215,4.095505e+13,0.351227,4.379215e+13,0.375558,3,8,1506.109164
1,101,1.773194e+09,9.780497e+06,2.310549e+14,0.066125,1.603531e+15,0.458911,1.659621e+15,0.474964,2,20,4080.489217
2,201,1.276751e+09,6.399131e+06,1.626690e+14,0.190067,3.339732e+14,0.390223,3.592098e+14,0.419710,2,7,2753.731956
3,301,1.248402e+09,6.154172e+06,1.677217e+14,0.217509,2.883583e+14,0.373957,3.150207e+14,0.408534,2,10,3050.673503
4,401,1.923074e+09,9.597811e+06,2.995687e+14,0.135017,9.553686e+14,0.430588,9.638181e+14,0.434396,5,19,4263.379274
...,...,...,...,...,...,...,...,...,...,...,...,...
866,86601,5.130087e+08,3.291605e+06,4.094854e+13,0.249575,5.776489e+13,0.352068,6.535983e+13,0.398358,2,8,2225.571221
867,86701,1.239189e+09,5.935591e+06,1.894589e+14,0.263351,2.357486e+14,0.327695,2.942071e+14,0.408953,6,12,2141.371047
868,86801,1.819499e+09,1.075195e+07,4.041112e+14,0.092490,1.871606e+15,0.428357,2.093548e+15,0.479153,4,26,5088.846446
869,86901,1.429212e+09,7.831558e+06,1.718710e+14,0.112398,6.337101e+14,0.414427,7.235429e+14,0.473175,5,21,3224.393651


In [1]:
from util.mesh import MeshProcessor
mp = MeshProcessor(path = "https://janelia-cosem-datasets.s3.amazonaws.com/jrc_mus-liver/neuroglancer/mesh/mito_seg",lod = 0, min_branch_length = 100, use_skeletons=True)
mp.process_mesh(id=4701).compute()

/Users/ackermand/miniconda3/envs/mesh_playground/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'id': 4701,
 'volume': 976077175.7379265,
 'surface_area': 5138911.456448785,
 'principal_inertia_component_0': -43511428473223.27,
 'principal_inertia_component_normalized_0': -0.17493239472485378,
 'principal_inertia_component_1': 74366594697898.28,
 'principal_inertia_component_normalized_1': 0.29898182970576836,
 'principal_inertia_component_2': 217877657342173.03,
 'principal_inertia_component_normalized_2': 0.8759505650190855,
 'num_fragments': 5,
 'num_branches': 13,
 'longest_path': 2413.8660331629953}

In [2]:
import trimesh
import numpy as np
mesh = trimesh.load_mesh("3201.ply")
print(mesh.is_watertight)
print(mesh.volume, mesh.area, mesh.principal_inertia_components)
mesh.apply_transform(mesh.principal_inertia_transform)
scene = trimesh.Scene()
scene.add_geometry(mesh)
axis = trimesh.creation.axis(origin_color=[1., 0, 0])
scene.add_geometry(axis)
scene.show()


False
1124452527.892105 6172824.167490382 [1.18360065e+14 2.97276792e+14 3.31818168e+14]


In [10]:
temp = df[["principal_inertia_component_0","principal_inertia_component_1","principal_inertia_component_1"]].to_numpy()>=0
temp.all()
_ = mp.get_mesh(id=4701).export("4701.ply")

In [ ]:
from sklearn.decomposition import PCA
row_norms = np.sum(momenta,axis=1)
normalized_momenta = momenta/row_norms[:,np.newaxis]
pca = PCA(n_components=3)
pca.fit(normalized_momenta)
transformed = pca.transform(normalized_momenta)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot()#projection='3d')
ax.scatter(normalized_momenta[:,2],normalized_momenta[:,0])
print(normalized_momenta[280,:],normalized_momenta[44,:],normalized_momenta[200,:])

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
data = df[["longest_path","axis_momenta_normalized_0","axis_momenta_normalized_1","axis_momenta_normalized_2"]].to_numpy()
data = StandardScaler().fit(data).transform(data)
kmeans = KMeans(n_clusters=3, random_state=0).fit(data)
for label in range(5):
    print([i+1 for i, x in enumerate(kmeans.labels_==label) if x])

In [ ]:
data[279],data[225]